In [1]:
import numpy as np
import toppra.dracula as drac
import toppra as ta
import toppra.algorithm as algo
from toppra import constraint
from toppra.dracula import RunTopp, V_MAX, A_MAX, _check_waypoints

In [2]:
import re
import ast
import numpy as np

def parse_str_arr(s):
    a = re.sub(r"([^[])\s+([^]])", r"\1, \2", s)
    return np.array(ast.literal_eval(a))

In [14]:
waypts = np.loadtxt('test_waypts_4.txt')
coeff = 1
vlim = coeff * np.vstack([-V_MAX, V_MAX]).T
alim = coeff * np.vstack([-A_MAX, A_MAX]).T

In [10]:
min_pair_dist, t_sum = _check_waypoints(waypts, vlim)

In [11]:
t_sum

0.064046801532567

In [103]:
N_samples = waypts.shape[0]
min_pair_dist, t_sum = _check_waypoints(waypts, vlim)
x = np.linspace(0, 0.15, N_samples)  # magic number
path = ta.SplineInterpolator(x, waypts.copy(), bc_type="clamped")
pc_vel = constraint.JointVelocityConstraint(vlim)
pc_acc = constraint.JointAccelerationConstraint(
    alim, discretization_scheme=constraint.DiscretizationType.Interpolation
)
instance = algo.TOPPRA(
    [pc_vel, pc_acc],
    path,
    # gridpoints=gridpoints,
    solver_wrapper="seidel",
)
jnt_traj = instance.compute_trajectory(0, 0)
jnt_traj.cspl.x

 INFO [algorithm.py : 100] No gridpoint specified. Automatically choose a gridpoint with 82 points
 INFO [algorithm.py : 185] Successfully parametrize path. Duration: 236.208, previously 0.150)
 INFO [algorithm.py : 186] Finish parametrization in 0.006 secs


array([0.00000000e+00, 3.16199586e-02, 4.60807571e-02, 6.98534902e-02,
       8.96981238e-02, 1.09219610e-01, 1.19978013e-01, 1.31548412e-01,
       1.43994573e-01, 1.57206783e-01, 1.71047139e-01, 1.85690955e-01,
       2.00384167e-01, 2.14353549e-01, 2.40340844e-01, 2.63714702e-01,
       2.86547900e-01, 2.98676786e-01, 3.11448229e-01, 3.24832863e-01,
       3.38772825e-01, 3.52667365e-01, 3.65999132e-01, 3.78726769e-01,
       3.90799480e-01, 4.02279883e-01, 4.13126165e-01, 4.32822977e-01,
       4.49881651e-01, 4.66819611e-01, 4.76227395e-01, 4.85987430e-01,
       4.95860630e-01, 5.05312706e-01, 5.14130230e-01, 5.22532177e-01,
       5.38172225e-01, 5.52096914e-01, 5.78110499e-01, 5.92391505e-01,
       6.05885490e-01, 6.17255324e-01, 6.27890502e-01, 6.38366510e-01,
       6.49605584e-01, 6.62253222e-01, 6.76260358e-01, 6.90817410e-01,
       7.04814861e-01, 7.31597208e-01, 7.45523006e-01, 7.60113422e-01,
       7.74783405e-01, 7.87926466e-01, 8.08726169e-01, 8.41283647e-01,
      

In [113]:
cs = jnt_traj.cspl
x = cs.x
vlim = vlim[:, 1]

In [114]:
vlim

array([1.0875, 1.0875, 1.0875, 1.0875, 1.305 , 1.305 , 1.305 ])

In [119]:
(np.abs(cs.derivative(1)(x))/vlim > 1).any()

False

In [ ]:
data = np.load('/data/toppra/20201204T175646.794406+0000.npz')
waypts = data['waypts']
vlim = data['vlim']
alim = data['alim']
vmax = vlim[:, 1]
amax = alim[:, 1]

In [25]:
waypts = np.array([
[ 0.1088, -1.5201,  1.1266, -2.6961,  1.1851,  1.5092, -1.8498],
[-0.5517, -0.5657,  1.7334, -1.8182,  0.5651,  1.5971, -1.1968],
])
vmax = np.array([0.54375, 0.54375, 0.54375, 0.54375,  0.6525,  0.6525,  0.6525])
amax = np.array([ 3.75, 1.875,   2.5, 3.125,  3.75,     5,     5])
vlim = np.vstack([-vmax, vmax]).T
alim = np.vstack([-amax, amax]).T

In [ ]:
cs = RunTopp(waypts, vlim, alim, return_cs=True, verify_lims=True)

In [95]:
order = 1
lim = vlim*0.893
deriv = cs.derivative(order)(cs.x)
deriv[13, 1] *= -1  # debug
i, j = np.where(~((lim[:, 0] < deriv) & (deriv < lim[:, 1])))
signed_lim = np.where(deriv > 0, lim[:, 1], lim[:, 0])
excess = np.sign(deriv) * (deriv - signed_lim)  # want the +ve entries
excess_percent = excess / np.abs(signed_lim)  # want +ve entries

In [85]:
(np.sign(deriv) * (deriv - signed_lim))[13]

array([-1.49205744e-01,  4.64335721e-04, -1.76552744e-01, -3.84936809e-02,
       -2.66944326e-01, -5.37918975e-01, -2.50138907e-01])

In [83]:
(deriv - signed_lim)[13]

array([ 1.49205744e-01, -4.64335721e-04, -1.76552744e-01, -3.84936809e-02,
        2.66944326e-01, -5.37918975e-01, -2.50138907e-01])

In [64]:
lim

array([[-0.48556875,  0.48556875],
       [-0.48556875,  0.48556875],
       [-0.48556875,  0.48556875],
       [-0.48556875,  0.48556875],
       [-0.5826825 ,  0.5826825 ],
       [-0.5826825 ,  0.5826825 ],
       [-0.5826825 ,  0.5826825 ]])

In [5]:
vmax - np.abs(cs.derivative(1)(cs.x))[21,:]

array([ 4.67432654e-01, -9.83477545e-05,  4.53298958e-01,  2.05413889e-01,
        6.41458630e-01,  3.03419892e-01,  4.19086688e-01])

In [7]:
cs = RunTopp(waypts, vlim, alim, return_cspl=True)

 INFO [__init__.py : 60] Debug environment detected, input data saved to: /data/toppra/20201204T175318.489327+0000.npz
 INFO [algorithm.py : 100] No gridpoint specified. Automatically choose a gridpoint with 73 points
 INFO [algorithm.py : 185] Successfully parametrize path. Duration: 1.377, previously 0.217)
 INFO [algorithm.py : 186] Finish parametrization in 0.012 secs
 INFO [__init__.py : 189] Returning optimised cubic spline trajectory of 77 knots


In [56]:
new_waypts = cs(cs.x)
new_waypts.shape

(9, 7)

In [57]:
new_cs = RunTopp(new_waypts, vlim, alim, return_cspl=True)

 INFO [__init__.py : 60] Debug environment detected, input data saved to: /data/toppra/20201204T160342.125263+0000.npz
WARNING [__init__.py : 85] Duplicates found in input waypoints. This is not recommended, especially for the beginning and the end of the trajectory. Toppra might throw a controllability exception. Attempting to optimise trajectory anyway...
 INFO [algorithm.py : 100] No gridpoint specified. Automatically choose a gridpoint with 5 points
 INFO [algorithm.py : 185] Successfully parametrize path. Duration: 0.095, previously 0.150)
 INFO [algorithm.py : 186] Finish parametrization in 0.002 secs
